### 1. data load

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
#  data load
df = pd.read_csv("./data/2019_반기보고서_01_재무상태표_연결_20200627.txt", sep = "\t", encoding = "cp949")

### 2. 연속형변수 변경 dtype 변경

In [2]:
def str_to_float(data):
    import numpy as np
    
    data["당기 반기말"] = data["당기 반기말"].str.replace(",", "")
    data["전기말"] = data["전기말"].str.replace(",", "")
#     data["전전기말"] = data["전전기말"].str.replace(",", "")
    data["당기 반기말"] = data["당기 반기말"].astype(np.float32)
    return data

df = str_to_float(df)

### 3. 불필요한 변수 제거

In [3]:
# 필요 없는 변수 제거
def delete_col(data):
#     data.drop("통화", axis = 1, inplace = True)
#     data.drop("Unnamed: 15", axis = 1, inplace = True)
    data.drop("전기말", axis = 1, inplace = True)
    data.drop("전전기말", axis = 1, inplace = True)
    data.drop("재무제표종류", axis = 1, inplace = True)
    return data

df = delete_col(df)

### 4. 항목명 변수 처리

In [4]:
def duplication_check(data):
    # 항목코드 중복 확인
    entity_코드 = []
    entity_명 = []
    without_entity_코드 = []
    without_entity_명 = []

    # 각 리스트에 코드 및 코드명 append(소문자 처리)
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            entity_코드.append(data["항목코드"][idx].lower())
            entity_명.append(data["항목명"][idx].lower())
        else:
            without_entity_코드.append(data["항목코드"][idx].lower())
            without_entity_명.append(data["항목명"][idx].lower())

    # !entity 코드 딕셔너리
    dic_without_entity = {re.findall("_\w{1,}", without_entity_코드[idx])[0].lower():[] for idx in range(len(without_entity_코드))}

    for i, j in zip(without_entity_코드, without_entity_명):
        dic_without_entity[re.findall("_\w{1,}", i)[0].lower()].append(j)

    check_without_entity = {i:len(set(j)) for i, j in dic_without_entity.items()}
    
    # entity 코드 딕셔너리
    dic_entity = {entity_코드[idx]:[] for idx in range(len(entity_코드))}
    
    for i, j in zip(entity_코드, entity_명):
        dic_entity[i].append(j)
        
    check_entity = {i:len(set(j)) for i, j in dic_entity.items()}
    
    return entity_코드, entity_명, without_entity_코드, without_entity_명, dic_without_entity, dic_entity

entity_코드, entity_명, without_entity_코드, without_entity_명, dic_without_entity, dic_entity = duplication_check(df)

### 5. 공시된 항목명, 항목코드 가져오기(금융감독원 재무제표 양식)

In [5]:
def extract_element_id(data):
    element_idx_lst_entity = []
    element_idx_lst_without_entity = []
    
    for idx in range(len(data)):
        if "entity" in data["항목코드"][idx]:
            element_idx_lst_entity.append(data["항목코드"][idx])
        else:
            element_idx_lst_without_entity.append(data["항목코드"][idx])
            
    entity = sorted(set(element_idx_lst_entity), reverse = True)
    without_entity = sorted(set(element_idx_lst_without_entity), reverse = True)
    
    return entity, without_entity

a, b = extract_element_id(df)

elementid = pd.read_excel("./data/재무제표양식.xlsx", encoding = "utf-8", sheet_name = "BS1")
# '한글 Label', 'Element ID'
df_element = elementid[['한글 Label', 'Element ID']]
df_element = df_element[df_element["Element ID"].notnull()]

df_element.columns = df_element.columns.str.replace(" ", "_")
lst_element = df_element["Element_ID"].unique()

re_lst_element = []
for i in lst_element:
    a = re.findall("_\w{1,}", i)[0].lower()
    re_lst_element.append(a)
 
dic_element = {i:[] for i in re_lst_element}
for i, j in zip(re_lst_element, df_element["한글_Label"]):
    dic_element[i].append(j)

### 5-1. 항목명 entity 값 처리

In [6]:
# 항목명 전치리

# 1) _가진 항목명 추출
lst_ = []
for i in range(len(df)):
    if "_" in df.loc[i, "항목명"]:
        lst_.append(i)
        
def preprocessing(x):
    a = x.replace(" ", "")
    a = a.replace("_", "")
    a = a.replace("[", "")
    a = a.replace("]", "")
    a = a.replace("(", "")
    a = a.replace(")", "")
    a = a.replace(".", "")
    a = re.sub("[ⅠⅡⅢⅣⅤIII]", "", a)
    a = re.sub("[1-9]", "", a)
    
    
    return a

df["항목명"] = df["항목명"].agg(preprocessing)

lst_idx = []
for i in range(len(df)):
    if "총계" in df["항목명"][i] and "entity" in df["항목코드"][i]:
        lst_idx.append(i)
        
for i in lst_idx:
    if df["항목명"][i] == '부채총계':
        df["항목코드"][i] = "ifrs-full_Liabilities"
        
    elif df["항목명"][i] == '자본과부채의총계':
        df["항목코드"][i] = "ifrs-full_EquityAndLiabilities"
        
    elif df["항목명"][i] == '자산총계':
        df["항목코드"][i] = "ifrs-full_Assets"
        
    else:
        df["항목코드"][i] = "ifrs-full_Equity"

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launche

### 6. entity 제외 항목명 변수 전처리

In [7]:
lst_idx = []
for idx in range(len(df)):
    try:
        if "entity" not in df["항목코드"][idx]:
            df["항목명"][idx] = dic_element[re.findall("_\w{1,}", df["항목코드"][idx])[0].lower()][0]
    except Exception as e:
        lst_idx.append(idx)
        
        
elementid = pd.read_excel("./data/재무제표양식.xlsx", encoding = "utf-8", sheet_name = "BS1")
# '한글 Label', 'Element ID'
df_element = elementid[['한글 Label', 'Element ID']]
df_element = df_element[df_element["Element ID"].notnull()]

df_element.columns = df_element.columns.str.replace(" ", "_")
lst_element = df_element["Element_ID"].unique()

re_lst_element = []
for i in lst_element:
    a = re.findall("_\w{1,}", i)[0].lower()
    re_lst_element.append(a)
 
dic_elementid = {(((i.replace("_", "")).replace(" ", "")).replace("[", "")).replace("]", ""):j 
                 for i, j in zip(df_element["한글_Label"], df_element["Element_ID"])}

idx_entity = []
for idx in range(len(df)):
    if "entity" in df.loc[idx, "항목코드"]:
        idx_entity.append(idx)

# entity 포함 항목코드 항목코드 전처리
no_preprocessing_entity_idx = []
for idx in idx_entity:
    try:
        df.loc[idx, "항목코드"] = dic_elementid[df.loc[idx, "항목명"]]

    except Exception as e:
        no_preprocessing_entity_idx.append(idx)
        
# 한번더 전처리
for idx in range(len(df)):
    if "entity" not in df["항목코드"][idx]:
        df["항목명"][idx] = dic_element[re.findall("_\w{1,}", df["항목코드"][idx])[0].lower()][0]
        

# 항목명이 같은 entity 변수 전처리
lst = []
for idx in range(len(df)):
    if "entity" in df.loc[idx, "항목코드"]:
        lst.append(idx)
        
no_preprocessing = []
for idx in lst:
    try:
        df.loc[idx, "항목코드"] = dic_elementid[df.loc[idx, "항목명"]]
    except Exception as e:
        no_preprocessing.append(idx)
        
        
# 새 기준 양식 불러오기
new_criterion = pd.read_excel("./data/개선코드표.xlsx", encoding = "utf-8", sheet_name = "2")
new_criterion

dic_re_code = {re.findall("_\w{1,}",i)[0]:j for i,j in zip(new_criterion["항목코드"], new_criterion["개선항목코드"])}
dic_re_name = {re.findall("_\w{1,}",i)[0]:j for i,j in zip(new_criterion["항목코드"], new_criterion["개선항목명"])}

# 새기준 양식으로 항목명 
for idx in range(len(df)):
    if "entity" not in df.loc[idx, "항목코드"]:
        df.loc[idx, "항목명"] = dic_re_name[re.findall("_\w{1,}", df.loc[idx, "항목코드"])[0]]
        df.loc[idx, "항목코드"] = dic_re_code[re.findall("_\w{1,}", df.loc[idx, "항목코드"])[0]]
        
# 필요 없는 변수 제거
index = df[df["항목명"] == "지우기"].index
df.drop(index, inplace = True)
df.reset_index(drop = True, inplace = True)

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### 7. 정제된 데이터 프레임 만들기

In [8]:
df.head()

,종목코드,회사명,시장구분,업종,업종명,결산월,결산기준일,보고서종류,통화,항목코드,항목명,당기 반기말,Unnamed: 15
0,[060310],3S,코스닥시장상장법인,292,특수 목적용 기계 제조업,3,2019-09-30,반기보고서,KRW,ifrs-full_AssetsAbstract,자산 [abstract],NaN,NaN
1,[060310],3S,코스닥시장상장법인,292,특수 목적용 기계 제조업,3,2019-09-30,반기보고서,KRW,ifrs-full_CurrentAssets,유동자산,1.709275e+10,NaN
2,[060310],3S,코스닥시장상장법인,292,특수 목적용 기계 제조업,3,2019-09-30,반기보고서,KRW,ifrs-full_CashAndCashEquivalents,현금및현금성자산,8.833444e+09,NaN
3,[060310],3S,코스닥시장상장법인,292,특수 목적용 기계 제조업,3,2019-09-30,반기보고서,KRW,sh_full_FinancialAsset_Current,금융자산(유동),6.114115e+07,NaN
4,[060310],3S,코스닥시장상장법인,292,특수 목적용 기계 제조업,3,2019-09-30,반기보고서,KRW,ifrs-full_TradeAndOtherCurrentReceivables,매출채권 및 기타유동채권,2.232528e+09,NaN


In [30]:
# DB 테이블에 적용할 D.F
# 회사이름 추출
lst_company = df["회사명"].unique()
element_value = list(new_criterion[new_criterion["개선항목명"] != "지우기"]["개선항목명"])
    
# columns
col = list(df.columns[0:9]) + element_value
check_col = list(dic_element.keys())

#  해당년도 데이터 행길이
company_dic = {i:[] for i in lst_company}
day = []

for i in lst_company:
    company_dic[i].append(df[df["회사명"] == i]["결산기준일"].unique())
    day.append(len(df[df["회사명"] == i]["결산기준일"].unique()))
    
row = sum(day)

bon = np.zeros([row,len(col)]) + np.NAN

# D.F
last_df = pd.DataFrame(bon, columns = col)

In [ ]:
last_df

### 8. 데이터 입력

In [22]:
dic_standard = {i:list(df[df["회사명"] == i]["결산기준일"].unique()) for i in lst_company}
last_company = []
last_standard  = []
for key, value in dic_standard.items():
    for index in range(len(value)):
        last_company.append(key)
        last_standard.append(value[index])
    
cnt = 0

for idx in range(len(last_company)):

    a = df[(df["회사명"] == last_company[idx]) & (df["결산기준일"] == last_standard[idx])]
    a.reset_index(drop = True, inplace = True)
    for i in range(9):
        last_df.iloc[cnt, i] = a.iloc[0, i]

    for idx3 in range(len(a)):
        if "entity" not in a["항목코드"][idx3]:
            loc = a["항목명"][idx3]
            last_df.loc[cnt, loc] = a["당기 반기말"][idx3]
    cnt += 1

In [23]:
df2019 = pd.read_csv("./data/2017_2019_without_entity.csv", encoding = "euc-kr")

In [24]:
df2019.shape

(5076, 362)

In [27]:
last_df

,종목코드,회사명,시장구분,업종,업종명,결산월,결산기준일,보고서종류,통화,재무상태표 [abstract],...,현금흐름위험회피파생상품평가손실,지분법기타포괄손익변동,이익잉여금(결손금),이익준비금,기타법정적립금,임의적립금,미처분이익잉여금(미처리결손금),비지배지분,자본총계,자본과부채총계
0,[060310],3S,코스닥시장상장법인,292.0,특수 목적용 기계 제조업,3.0,2019-09-30,반기보고서,KRW,NaN,...,NaN,NaN,-3.117097e+10,NaN,NaN,NaN,NaN,NaN,3.487044e+10,5.161758e+10
1,[095570],AJ네트웍스,유가증권시장상장법인,763.0,산업용 기계 및 장비 임대업,12.0,2019-06-30,반기보고서,KRW,NaN,...,NaN,NaN,2.488425e+11,NaN,NaN,NaN,NaN,NaN,3.984555e+11,1.740845e+12
2,[006840],AK홀딩스,유가증권시장상장법인,649.0,기타 금융업,12.0,2019-06-30,반기보고서,KRW,NaN,...,NaN,NaN,5.178931e+11,NaN,NaN,NaN,NaN,NaN,1.507604e+12,4.269534e+12
3,[054620],APS홀딩스,코스닥시장상장법인,649.0,기타 금융업,12.0,2019-06-30,반기보고서,KRW,NaN,...,NaN,NaN,7.467813e+11,NaN,NaN,NaN,NaN,NaN,2.295536e+11,2.828088e+11
4,[265520],AP시스템,코스닥시장상장법인,292.0,특수 목적용 기계 제조업,12.0,2019-06-30,반기보고서,KRW,NaN,...,NaN,NaN,4.227868e+10,NaN,NaN,NaN,NaN,NaN,1.104254e+11,3.934398e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,[069260],휴켐스,유가증권시장상장법인,204.0,기타 화학제품 제조업,12.0,2019-06-30,반기보고서,KRW,NaN,...,NaN,NaN,4.502304e+11,NaN,NaN,NaN,NaN,NaN,6.581130e+11,8.815452e+11
1634,[010240],흥국,코스닥시장상장법인,292.0,특수 목적용 기계 제조업,12.0,2019-06-30,반기보고서,KRW,NaN,...,NaN,NaN,5.631928e+10,NaN,NaN,NaN,NaN,NaN,6.434972e+10,8.804965e+10
1635,[189980],흥국에프엔비,코스닥시장상장법인,112.0,비알코올음료 및 얼음 제조업,12.0,2019-06-30,반기보고서,KRW,NaN,...,NaN,NaN,3.374839e+10,NaN,NaN,NaN,NaN,NaN,6.213871e+10,8.862392e+10
1636,[003280],흥아해운,유가증권시장상장법인,501.0,해상 운송업,12.0,2019-06-30,반기보고서,KRW,NaN,...,NaN,NaN,-2.285760e+10,NaN,NaN,NaN,NaN,NaN,4.373892e+10,8.008379e+11


In [12]:
col2019 = df2019.columns
col = last_df.columns

last_df.columns = col2019

ValueError: Length mismatch: Expected axis has 535 elements, new values have 362 elements

In [17]:
real_df = pd.concat([df2019, last_df]).reset_index(drop = True)

In [18]:
real_df.to_csv("./data/분기_반기.csv", encoding = "euc=kr", index = False)